In [0]:
# FUNÇÕES UTILIZADAS ==> (when + otherwise) / (orderBy) / (groupBy)

In [0]:
# IMPORTANDO BIBLIOTECAS 

import pyspark.sql.functions
from pyspark.sql.types import StructType, StructField , StringType , IntegerType , FloatType
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [0]:
spark = (SparkSession.builder
        .master("local(*)")
        .appName('pyspark-4')
        .getOrCreate()
        )

In [0]:
# CRIANDO DATA FRAME 

dados = (

    ['1','Celta','Chevrolet',2004,18000,"Não"],
    ['2','Gol','volkswagen',2020,46320,"Sim"],
    ['3','Corsa Classic', 'Chevrolet',2008,15000,"Sim"],
    ['4','Palio','Fiat',1998,8900,"Não"],
    ['5','ASX','Mitsubishi',2016,65000,"Sim"],
    ['6','Corolla','Toyota',2018,89990,"Sim"],
    ['7','Civic','Honda',2019,89990,"Sim"],
    ['8','Eco Sport','Ford',2017,69299,"Sim"],
    ['9','Onix','Chevrolet',2020,58990,"Sim"]

)

In [0]:
colunas = (StructType([
    StructField('cod' , StringType(), True),
    StructField('Carro' , StringType(), True),
    StructField('Marca' , StringType(), True),
    StructField('Ano' , IntegerType(), True),
    StructField('Preco' , IntegerType(), True),
    StructField('Flex' , StringType(), True)
]))

In [0]:
# DATA FRAME FINAL 

df = spark.createDataFrame(data = dados , schema = colunas)

df.display()

cod,Carro,Marca,Ano,Preco,Flex
1,Celta,Chevrolet,2004,18000,Não
2,Gol,volkswagen,2020,46320,Sim
3,Corsa Classic,Chevrolet,2008,15000,Sim
4,Palio,Fiat,1998,8900,Não
5,ASX,Mitsubishi,2016,65000,Sim
6,Corolla,Toyota,2018,89990,Sim
7,Civic,Honda,2019,89990,Sim
8,Eco Sport,Ford,2017,69299,Sim
9,Onix,Chevrolet,2020,58990,Sim


In [0]:
# 1) CRIE UMA COLUNA (BOM PARA TER) SOMENTE PARA CARROS (FLEX) 

df1 = df.withColumn('Bom para ter' , when(col('Flex') == 'Sim' , "OK").otherwise('NOK'))

df1.show()

+---+-------------+----------+----+-----+----+------------+
|cod|        Carro|     Marca| Ano|Preco|Flex|Bom para ter|
+---+-------------+----------+----+-----+----+------------+
|  1|        Celta| Chevrolet|2004|18000| Não|         NOK|
|  2|          Gol|volkswagen|2020|46320| Sim|          OK|
|  3|Corsa Classic| Chevrolet|2008|15000| Sim|          OK|
|  4|        Palio|      Fiat|1998| 8900| Não|         NOK|
|  5|          ASX|Mitsubishi|2016|65000| Sim|          OK|
|  6|      Corolla|    Toyota|2018|89990| Sim|          OK|
|  7|        Civic|     Honda|2019|89990| Sim|          OK|
|  8|    Eco Sport|      Ford|2017|69299| Sim|          OK|
|  9|         Onix| Chevrolet|2020|58990| Sim|          OK|
+---+-------------+----------+----+-----+----+------------+



In [0]:
# 2) SELECIONE SOMENTE OS CARROS DA (CHEVROLET) QUE SÃO (BOM PARA TER)

df1.filter(col('Marca') == 'Chevrolet').filter(col('Bom para ter') == 'OK').show()

+---+-------------+---------+----+-----+----+------------+
|cod|        Carro|    Marca| Ano|Preco|Flex|Bom para ter|
+---+-------------+---------+----+-----+----+------------+
|  3|Corsa Classic|Chevrolet|2008|15000| Sim|          OK|
|  9|         Onix|Chevrolet|2020|58990| Sim|          OK|
+---+-------------+---------+----+-----+----+------------+



In [0]:
# 3) ENCONTRE OS CARROS QUE NÃO SÃO (FLEX)

df1.filter(col('Flex') == "Não").show()

+---+-----+---------+----+-----+----+------------+
|cod|Carro|    Marca| Ano|Preco|Flex|Bom para ter|
+---+-----+---------+----+-----+----+------------+
|  1|Celta|Chevrolet|2004|18000| Não|         NOK|
|  4|Palio|     Fiat|1998| 8900| Não|         NOK|
+---+-----+---------+----+-----+----+------------+



In [0]:
df1.printSchema()

root
 |-- cod: string (nullable = true)
 |-- Carro: string (nullable = true)
 |-- Marca: string (nullable = true)
 |-- Ano: integer (nullable = true)
 |-- Preco: integer (nullable = true)
 |-- Flex: string (nullable = true)
 |-- Bom para ter: string (nullable = false)



In [0]:
# 4) ENCONTRE CARROS (BOM PARA TER) QUE TENHAM ANO MAIOR QUE 2016 ; CLASSIFIQUE OS CARROS POR ORDEM ALFABETICA E MOSTRE A QNT

test = df1.filter(col('Ano') > 2016).filter(col('Bom para ter') == 'OK')

test.show()

# CLASSIFIQUE OS CARROS POR ORDEM ALFABETICA E MOSTRE A QNT

#teste = df1.filter(col('Ano') > 2016).filter(col('Bom para ter') == 'OK').orderBy(col('Carro').asc())

teste = df1.filter(col('Ano') > 2016).filter(col('Bom para ter') == 'OK').groupBy('Carro').count().orderBy(col('Carro'))

teste.show()




+---+---------+----------+----+-----+----+------------+
|cod|    Carro|     Marca| Ano|Preco|Flex|Bom para ter|
+---+---------+----------+----+-----+----+------------+
|  2|      Gol|volkswagen|2020|46320| Sim|          OK|
|  6|  Corolla|    Toyota|2018|89990| Sim|          OK|
|  7|    Civic|     Honda|2019|89990| Sim|          OK|
|  8|Eco Sport|      Ford|2017|69299| Sim|          OK|
|  9|     Onix| Chevrolet|2020|58990| Sim|          OK|
+---+---------+----------+----+-----+----+------------+

+---------+-----+
|    Carro|count|
+---------+-----+
|    Civic|    1|
|  Corolla|    1|
|Eco Sport|    1|
|      Gol|    1|
|     Onix|    1|
+---------+-----+

